**Single table column **transformation****

In [ ]:
# List contents of bronze directory to verify our source
dbutils.fs.ls('mnt/bronze/SalesLT/')

In [ ]:
# List contents of silver directory to verify our destination
dbutils.fs.ls('mnt/silver/')

In [ ]:
# Read the Address parquet file from bronze layer into a DataFrame
df = spark.read.format('parquet').load('/mnt/bronze/SalesLT/Address/Address.parquet')

In [ ]:
# Display the DataFrame to verify the data was loaded correctly
display(df)

In [ ]:
# Import necessary PySpark functions for date transformation
from pyspark.sql.functions import from_utc_timestamp, date_format
from pyspark.sql.types import TimestampType

# Transform ModifiedDate column:
# 1. Cast the column to TimestampType
# 2. Convert from UTC timezone
# 3. Format the date as YYYY-MM-DD
df = df.withColumn('ModifiedDate', date_format(from_utc_timestamp(df['ModifiedDate'].cast(TimestampType()), 'UTC'), 'yyyy-MM-dd'))

In [ ]:
# Display the DataFrame to verify the date transformation
display(df)

**Date transformation for all the tables**

In [ ]:
# Initialize empty list to store table names
table_name = []

# Get all table names from the bronze layer
# The split('/')[0] removes the trailing slash from directory names
for i in dbutils.fs.ls('mnt/bronze/SalesLT/'):
    table_name.append(i.name.split('/')[0])

# Display list of tables that will be processed
table_name

In [ ]:
# Import required functions (if running in a new session)
from pyspark.sql.functions import from_utc_timestamp, date_format
from pyspark.sql.types import TimestampType

# Process each table in the bronze layer
for i in table_name:
    # Load the parquet file into a DataFrame
    path = '/mnt/bronze/SalesLT/' + i + '/' + i + '.parquet'

    # Read the parquet file
    df = spark.read.format('parquet').load(path)

    # Get all column names in the current table
    column = df.columns

    # Look for date columns and transform them
    for col in column:

        # Check if the column name contains 'Date' or 'date'
        if "Date" in col or "date" in col:

            # Transform the date column:
            # 1. Cast to TimestampType
            # 2. Convert from UTC
            # 3. Format as YYYY-MM-DD
            df = df.withColumn(col, date_format(from_utc_timestamp(df[col].cast(TimestampType()), "UTC"), "yyyy-MM-dd"))

    # Construct the output path in the silver layer
    output_path = '/mnt/silver/SalesLT/' + i + '/'

    # Write the transformed DataFrame to silver layer:
    # - Using delta format for better data management
    # - Overwrite mode replaces existing data
    df.write.format('delta').mode('overwrite').save(output_path)


In [ ]:
# Display the final transformed DataFrame (last table processed)
display(df)